## Tensorflow的常见Metrics评价函数

***含义：***  
评价函数Metrics用于评估当前训练模型的性能

***和损失函数Loss Function相同的地方：***  
评价函数和 损失函数 相似，函数声明都是func(y_true, y_pred)

***和损失函数Loss Function不同的地方：***  
1. 评价函数的结果不会用于训练过程中
2. 评价函数是有状态的

跟损失函数又不一样，评价函数会跨训练batch记录数据计算指标：  
比如为了计算Accuracy，会维护total变量记录见过的记录数，维护count变量记录正确匹配的记录数  
计算公式：Accuracy = count/total

* 使用update_state函数更新数据状态（更新count、total）
* 使用result函数查询metric数值（查询迄今为止的最终数值count/total）
* 使用reset_states清空状态（total=count=0）

In [1]:
import tensorflow as tf
import numpy as np

### 1. 评价函数Metrics的使用

可以传递已有的评价函数名称，或者传递一个自定义的TensorFlow函数来使用

In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [3]:
# 方法1：传入字符串名称
model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy', 'auc'])

In [4]:
# 方法2：传入函数名列表
model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.AUC()])

### 2. 编写自己的Metrics评价函数

In [5]:
def my_metric_fn(y_true, y_pred):
    """自己的Metrics评价函数"""
    return tf.reduce_mean(tf.square(y_true - y_pred), axis=-1)

In [6]:
# 做下测试
my_metric_fn(np.array([0, 0, 1, 1]), np.array([0, 0.5, 0.3, 0.9]))

<tf.Tensor: shape=(), dtype=float64, numpy=0.1875>

In [7]:
# 使用方式
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[my_metric_fn])

### 3. 常见的分类评价函数

#### 准确率（Accuracy）

In [8]:
m = tf.keras.metrics.Accuracy()
m.update_state([[1], [2], [3], [4]], [[0], [2], [3], [4]])
m.result().numpy()

0.75

#### AUC（Area Under Curve）

In [9]:
m = tf.keras.metrics.AUC()
m.update_state([0, 0, 1, 1], [0, 0.5, 0.3, 0.9])
m.result().numpy()

0.75

#### 精确率（Precision）

In [10]:
m = tf.keras.metrics.Precision()
m.update_state([0, 1, 1, 1], [1, 0, 1, 1])
m.result().numpy()

0.6666667

#### 召回率（Recall）

In [11]:
m = tf.keras.metrics.Recall()
m.update_state([0, 1, 1, 1], [1, 0, 1, 1])
m.result().numpy()

0.6666667

### 4. 常见的回归评价函数

#### MSE（Mean Square Error）均方误差

In [12]:
m = tf.keras.metrics.MeanSquaredError()
m.update_state([[0, 1], [0, 0]], [[1, 1], [0, 0]])
m.result().numpy()

0.25

#### RMSE（Root Mean Square Error）均方根误差

In [13]:
m = tf.keras.metrics.RootMeanSquaredError()
m.update_state([[0, 1], [0, 0]], [[1, 1], [0, 0]])
m.result().numpy()

0.5

#### MAE（Mean Absolute Error）平均绝对误差

In [14]:
m = tf.keras.metrics.MeanAbsoluteError()
m.update_state([[0, 1], [0, 0]], [[1, 1], [0, 0]])
m.result().numpy()

0.25